In [1]:
import pandas as pd
import geopandas as gpd
import os
import fiona

# Data sources
* p10: county designation, value, and size attributes from p10 in smelt (3/12 version), jurisdiction designation from the updated 'parcels_geography.csv' file with the new juri_id values from Mike
* b0: from smelt (3/12 version)
* p18: urbansim_parcels_v4_tbl from UrbanSim Data Review Option B v1d.gdb (https://mtcdrive.box.com/s/6xhdmzs5m9q0amzaz7etsej4lbw8jbik)
* b18: urbansim_buildings_v4_tbl.csv (https://mtcdrive.box.com/s/unodascuuukx7y8hiqf9m299bnr4it70)

In [91]:
cnty = {'Alameda': 1.0,
'Contra Costa': 13.0,
'Marin': 41.0,
'Napa': 55.0,
'San Francisco': 75.0,
'San Mateo': 81.0,
'Santa Clara': 85.0,
'Solano': 95.0,
'Sonoma': 97.0}
ctyMap = pd.DataFrame(cnty.items(), columns=['ctyName', 'ctyCode'])

juris_lookup = pd.read_csv(r'C:\Users\ywang\Documents\GitHub\bayarea_urbansim\data\census_id_to_name.csv')
juris_lookup2 = pd.read_csv(r'C:\Users\ywang\Documents\GitHub\petrale\zones\jurisdictions\jurisdictions.csv')
juris_lookup['name10'] = juris_lookup['name10'].apply(lambda x: x.lower().replace(' ','_'))
juris_lookup['name10'] = juris_lookup['name10'].apply(lambda x: x.replace('.',''))
juris_lookup.columns = ['jurisdiction_id', 'name10']
juris_lookup2['name2'] = juris_lookup2['name']
for i in range(juris_lookup2.shape[0]):
    if 'unincorporated_' in juris_lookup2.iloc[i,0]:
        juris_lookup2.iloc[i,5] = juris_lookup2.iloc[i,0].replace('unincorporated_','')+'_county'
jurisId = juris_lookup2.merge(juris_lookup, left_on = 'name2', right_on = 'name10', how = 'left')
jurisId.rename(columns={'name': 'juris_name'}, inplace=True)

## Parcels

### BASIS parcel (p18)

In [3]:
fiona.listlayers(r'C:\Users\ywang\Documents\ArcGIS Pro 2.5\WorkingFiles\UrbanSim Data Review Option B v1d.gdb')

['p10_boc_opt_b_v1d',
 'urbansim_boc_v4a_tbl',
 'p10_geo',
 'p18_geo',
 'p10_boc_opt_b_v1d_geo',
 'urbansim_parcels_v4_tbl',
 'p18_attr_tbl']

In [4]:
p18_raw = gpd.read_file(r'C:\Users\ywang\Documents\ArcGIS Pro 2.5\WorkingFiles\UrbanSim Data Review Option B v1d.gdb', layer='urbansim_parcels_v4_tbl')

In [125]:
p18 = pd.DataFrame(p18)
print(p18.shape)
print(len(p18.geom_id.unique()))

(2044385, 13)
2044385


In [126]:
chk_nan_id = p18.loc[(p18.county_id.isnull()) | (p18.jurisdiction.isnull())]
print('Num of parcels missing county or jurisdiction designation: ', chk_nan_id.shape[0])
chk_nan_value = p18.loc[p18.assessed_land_value.isnull()]
print('Num of parcels missing assessed land value: ', chk_nan_value.shape[0])
chk_nan_acre = p18.loc[p18.acres.isnull()]
print('Num of parcels mising acreage: ', chk_nan_acre.shape[0])

Num of parcels missing county or jurisdiction designation:  0
Num of parcels missing assessed land value:  87761
Num of parcels mising acreage:  0


In [127]:
def groupSum(data,group_field,sum_field,yr):
    df = data.groupby([group_field])[sum_field].sum()
    df.columns = [x + '_' + str(yr) for x in list(df)]
    return df

In [128]:
p18.jurisdiction = p18.jurisdiction.apply(lambda x: x.lower().replace(' ','_'))
p18_cty = groupSum(p18,'county', ['acres','assessed_land_value'],'18')
p18_jur = groupSum(p18,'jurisdiction',['acres','assessed_land_value'],'18')

#display(p18_cty)
#display(p18_jur)

### PBA40 parcel (p10)

In [206]:
#p10_raw = gpd.read_file(r'C:\Users\ywang\Documents\Python Scripts\BASIS\BASIS_PB18\inputs\p10_geo_shp.shp')
p10_raw = gpd.read_file(r'C:\Users\ywang\Documents\ArcGIS Pro 2.5\WorkingFiles\smelt\2020 03 12\smelt.gdb', layer='p10_tbl')

In [224]:
p10_juris = pd.read_csv(r'C:\Users\ywang\Documents\Python Scripts\BASIS\BASIS_PB18\inputs\p10_jurisID.csv')

In [233]:
p10 = p10_raw[['PARCEL_ID','DEVELOPMEN','LAND_VALUE','ACRES','COUNTY_ID']]
p10 = p10.merge(p10_juris, on = 'PARCEL_ID', how = 'left').merge(
    ctyMap, left_on = 'COUNTY_ID', right_on = 'ctyCode', how = 'left').merge(
    jurisId[['juris_name','juris']], left_on = 'juris_id', right_on = 'juris', how = 'left')
p10_cty = groupSum(p10,'ctyName',['LAND_VALUE', 'ACRES'],'10')
p10_jur = groupSum(p10,'juris_name',['LAND_VALUE', 'ACRES'],'10')

### Comparison

In [234]:
def diff(df, atts):
    atts_10 = [x + '_10' for x in atts]
    atts_18 = [x + '_18' for x in atts]
    for i in range(len(atts)):
        df[atts[i] +'_diff'] = df[atts_18[i]] - df[atts_10[i]]
        df[atts[i] +'_diff_pct'] = df[atts[i] +'_diff'] / df[atts_10[i]]
    for i in atts_10 + atts_18 + [x + '_diff' for x in atts]:
        df[i] = df[i].apply(lambda x: f'{int(x):,}')
    return df

def summaryTable(df,group,att):
    df2 = df[[group] + [att + '_10'] + [att + '_18'] + [att + '_diff'] + [att + '_diff_pct']]
    #df2.to_csv('results/' + att + '_' + group + '.csv', index = False)
    return (att + '_' + group, df2)

In [235]:
## Counth comparison
p_cty_concat = pd.concat([p10_cty,p18_cty],axis=1, join='outer').reset_index()
p_cty_concat.columns = ['county','land_value_10','acres_10','acres_18','land_value_18']
#display(p_cty_concat)

p_cty = diff(p_cty_concat, ['land_value','acres'])
#display(p_cty)

In [236]:
## Jurisdiction comparison
p_jur_concat = pd.concat([p10_jur,p18_jur],axis=1, join='outer').reset_index()
p_jur_concat.columns = ['jurisdiction','land_value_10','acres_10','acres_18','land_value_18']
display(p_jur_concat)

p_jur = diff(p_jur_concat, ['land_value','acres'])
#display(p_jur)

,jurisdiction,land_value_10,acres_10,acres_18,land_value_18
0,alameda,2.758287e+09,6576.815744,14057.17910,4.024631e+09
1,albany,5.454092e+08,2807.062862,2866.75197,7.886502e+08
2,american_canyon,3.040250e+08,3462.900238,2669.76540,7.926562e+08
3,antioch,2.540386e+09,15654.496344,15650.34329,3.365071e+09
4,atherton,3.906331e+09,2887.029852,2902.53638,6.858041e+09
...,...,...,...,...,...
104,vallejo,2.145679e+09,18923.391859,23323.14567,2.745208e+09
105,walnut_creek,6.018645e+09,11199.444875,11269.25087,7.481297e+09
106,windsor,1.144831e+09,4004.684711,3977.68215,1.485007e+09
107,woodside,2.815574e+09,6659.912290,6630.61805,4.067712e+09


In [273]:
writer = pd.ExcelWriter('results/BASIS_p18_20200401.xlsx', engine='xlsxwriter')

ls = []
name = []
dfs = {'county': p_cty, 'jurisdiction': p_jur}
atts = ['land_value','acres']
for i in list(dfs.keys()):
    for j in atts:
        result = summaryTable(dfs[i],i,j)
        ls.append(result[1])
        name.append(result[0])

for i in name:
    ls[name.index(i)].to_excel(writer, sheet_name = i)

writer.save()

In [274]:
p18.to_csv('results/p18.csv', index = False)
p10.to_csv('results/p10.csv', index = False)

## Buildings


### PBA40 building (b10)

In [238]:
b10_raw = gpd.read_file(r'C:\Users\ywang\Documents\ArcGIS Pro 2.5\WorkingFiles\smelt\2020 03 12\smelt.gdb', layer='b10')

In [252]:
b10 = b10_raw.drop(columns = ['geometry'])
b10 = b10.merge(p10[['PARCEL_ID','ctyName','juris_name']], left_on = 'parcel_id', right_on = 'PARCEL_ID', how = 'left')

In [253]:
b10.head()

,building_id,parcel_id,development_type_id,improvement_value,residential_units,residential_sqft,sqft_per_unit,non_residential_sqft,building_sqft,nonres_rent_per_sqft,...,year_built,redfin_sale_price,redfin_sale_year,redfin_home_type,costar_property_type,costar_rent,id,PARCEL_ID,ctyName,juris_name
0,1,742974,1,0.00,1,2029,2029.42425,0,2029.42425,0.0,...,1945,NaN,NaN,None,None,None,1,742974.0,Marin,fairfax
1,2,744961,1,0.00,1,2029,2029.42425,0,2029.42425,0.0,...,1965,NaN,NaN,None,None,None,2,744961.0,Marin,unincorporated_marin
2,3,1442641,1,53262.87,1,1568,1568.00000,0,1568.00000,0.0,...,1964,NaN,NaN,None,None,None,3,1442641.0,Santa Clara,santa_clara
3,4,190969,2,245000.00,0,0,1266.00000,1595,1266.00000,0.0,...,1992,340000.0,2003.0,Condo/Coop,None,None,4,190969.0,Alameda,unincorporated_alameda
4,5,308709,2,283500.00,0,0,1513.00000,1513,1513.00000,0.0,...,1978,442000.0,2004.0,Condo/Coop,None,None,5,308709.0,Alameda,fremont


In [254]:
b10_cty = groupSum(b10,'ctyName',['improvement_value','building_sqft','residential_units','non_residential_sqft'],'10')
b10_jur = groupSum(b10,'juris_name',['improvement_value','building_sqft','residential_units','non_residential_sqft'],'10')

### BASIS building (b18)

In [244]:
b18 = pd.read_csv(r'C:\Users\ywang\Documents\Python Scripts\BASIS\BASIS_PB18\inputs\urbansim_buildings_v4_tbl.csv')

In [250]:
b18 = b18.merge(p18[['geom_id','county_id', 'county','jurisdiction']], on = 'geom_id', how = 'left')
print(b18.shape)
b18.loc[b18.county_id.isnull()]

(2044418, 14)


,last_sale_price,assessed_building_value,year_built,residential_units,non_residential_sqft,building_sqft,last_sale_date,assessed_date,building_type,building_id,geom_id,county_id,county,jurisdiction


In [256]:
b18.jurisdiction = b18.jurisdiction.apply(lambda x: x.lower().replace(' ','_'))
b18_cty = groupSum(b18,'county',['assessed_building_value','building_sqft','residential_units','non_residential_sqft'],'18')
b18_jur = groupSum(b18,'jurisdiction',['assessed_building_value','building_sqft','residential_units','non_residential_sqft'],'18')

In [257]:
b18_cty

,assessed_building_value_18,building_sqft_18,residential_units_18,non_residential_sqft_18
county,,,,
Alameda,1.745541e+11,1.153557e+09,5.413170e+05,4.016977e+08
Contra Costa,1.198971e+11,7.934421e+08,1.605395e+06,1.471375e+08
Marin,3.881693e+10,2.107235e+08,1.005800e+05,0.000000e+00
Napa,1.991207e+10,9.569405e+07,1.938750e+04,2.320780e+07
San Francisco,1.183459e+11,5.391285e+08,1.895460e+05,1.873583e+08
San Mateo,9.908947e+10,1.085997e+09,2.854888e+05,4.267465e+07
Santa Clara,2.205049e+11,1.253122e+09,4.909385e+05,3.617783e+08
Solano,3.785838e+10,3.875441e+08,1.502112e+05,1.079605e+08
Sonoma,8.424780e+10,3.858217e+08,1.672805e+05,1.244249e+08


### Compare

In [266]:
## Counth comparison
b_cty_concat = pd.concat([b10_cty,b18_cty],axis=1, join='outer').reset_index()
b_cty_concat.columns = ['county','improvement_value_10','building_sqft_10','res_units_10','non_res_sqft_10',
                        'improvement_value_18','building_sqft_18','res_units_18','non_res_sqft_18']
display(b_cty_concat)

b_cty = diff(b_cty_concat, ['improvement_value','building_sqft','res_units','non_res_sqft'])
display(b_cty)

,county,improvement_value_10,building_sqft_10,res_units_10,non_res_sqft_10,improvement_value_18,building_sqft_18,res_units_18,non_res_sqft_18
0,Alameda,1.139750e+11,1.312090e+09,582594,558043969,1.745541e+11,1.153557e+09,5.413170e+05,4.016977e+08
1,Contra Costa,9.311274e+10,9.159326e+08,400508,252767718,1.198971e+11,7.934421e+08,1.605395e+06,1.471375e+08
2,Marin,2.346466e+10,2.825205e+08,111201,64595667,3.881693e+10,2.107235e+08,1.005800e+05,0.000000e+00
3,Napa,5.392976e+09,1.434065e+08,54884,53617752,1.991207e+10,9.569405e+07,1.938750e+04,2.320780e+07
4,San Francisco,7.262598e+10,8.762487e+08,376657,416112578,1.183459e+11,5.391285e+08,1.895460e+05,1.873583e+08
5,San Mateo,7.801676e+10,7.176125e+08,270966,202575180,9.908947e+10,1.085997e+09,2.854888e+05,4.267465e+07
6,Santa Clara,1.443857e+11,1.890054e+09,629953,590039030,2.205049e+11,1.253122e+09,4.909385e+05,3.617783e+08
7,Solano,2.686472e+10,3.477815e+08,152772,89468249,3.785838e+10,3.875441e+08,1.502112e+05,1.079605e+08
8,Sonoma,3.850960e+10,4.316477e+08,204490,124269753,8.424780e+10,3.858217e+08,1.672805e+05,1.244249e+08


,county,improvement_value_10,building_sqft_10,res_units_10,non_res_sqft_10,improvement_value_18,building_sqft_18,res_units_18,non_res_sqft_18,improvement_value_diff,improvement_value_diff_pct,building_sqft_diff,building_sqft_diff_pct,res_units_diff,res_units_diff_pct,non_res_sqft_diff,non_res_sqft_diff_pct
0,Alameda,"113,975,001,774","1,312,089,954","582,594","558,043,969","174,554,138,587","1,153,556,993","541,317","401,697,661","60,579,136,813",0.531512,"-158,532,961",-0.120825,"-41,277",-0.070850,"-156,346,308",-0.280168
1,Contra Costa,"93,112,741,287","915,932,607","400,508","252,767,718","119,897,134,125","793,442,141","1,605,394","147,137,517","26,784,392,838",0.287656,"-122,490,466",-0.133733,"1,204,886",3.008396,"-105,630,200",-0.417894
2,Marin,"23,464,655,156","282,520,457","111,201","64,595,667","38,816,928,134","210,723,520","100,580",0,"15,352,272,978",0.654272,"-71,796,937",-0.254130,"-10,621",-0.095512,"-64,595,667",-1.000000
3,Napa,"5,392,975,629","143,406,454","54,884","53,617,752","19,912,070,374","95,694,048","19,387","23,207,803","14,519,094,744",2.692223,"-47,712,405",-0.332708,"-35,496",-0.646755,"-30,409,948",-0.567162
4,San Francisco,"72,625,982,720","876,248,653","376,657","416,112,578","118,345,893,054","539,128,481","189,546","187,358,334","45,719,910,334",0.629526,"-337,120,171",-0.384731,"-187,111",-0.496768,"-228,754,243",-0.549741
5,San Mateo,"78,016,755,923","717,612,514","270,966","202,575,180","99,089,467,802","1,085,996,944","285,488","42,674,652","21,072,711,879",0.270105,"368,384,430",0.513347,"14,522",0.053597,"-159,900,527",-0.789339
6,Santa Clara,"144,385,670,015","1,890,054,275","629,953","590,039,030","220,504,863,340","1,253,121,501","490,938","361,778,317","76,119,193,324",0.527194,"-636,932,773",-0.336992,"-139,014",-0.220674,"-228,260,712",-0.386857
7,Solano,"26,864,715,250","347,781,501","152,772","89,468,249","37,858,375,334","387,544,099","150,211","107,960,532","10,993,660,084",0.409223,"39,762,598",0.114332,"-2,560",-0.016762,"18,492,283",0.206691
8,Sonoma,"38,509,600,570","431,647,738","204,490","124,269,753","84,247,800,076","385,821,684","167,280","124,424,926","45,738,199,506",1.187709,"-45,826,054",-0.106165,"-37,209",-0.181962,"155,173",0.001249


In [267]:
## Jurisdiction comparison
b_jur_concat = pd.concat([b10_jur,b18_jur],axis=1, join='outer').reset_index()
b_jur_concat.columns = ['jurisdiction','improvement_value_10','building_sqft_10','res_units_10','non_res_sqft_10',
                        'improvement_value_18','building_sqft_18','res_units_18','non_res_sqft_18']
display(b_jur_concat)

b_jur = diff(b_jur_concat, ['improvement_value','building_sqft','res_units','non_res_sqft'])
display(b_jur)


,jurisdiction,improvement_value_10,building_sqft_10,res_units_10,non_res_sqft_10,improvement_value_18,building_sqft_18,res_units_18,non_res_sqft_18
0,alameda,5.370707e+09,6.069406e+07,32056,20015176,7.922534e+09,4.820632e+07,30080.000000,9.642204e+06
1,albany,1.053810e+09,1.001083e+07,7779,2539429,1.604529e+09,9.004613e+06,5665.000000,2.264891e+06
2,american_canyon,3.040250e+08,1.933756e+07,5773,7620221,2.047428e+09,1.683769e+07,434.000000,6.004531e+06
3,antioch,5.899923e+09,7.640542e+07,34821,17003933,8.380102e+09,6.953155e+07,141212.333333,1.020848e+07
4,atherton,2.533024e+09,8.785455e+06,2523,1239883,4.298967e+09,2.349359e+07,2473.000000,0.000000e+00
...,...,...,...,...,...,...,...,...,...
104,vallejo,6.281622e+09,8.745682e+07,44357,17703716,8.560594e+09,1.150469e+08,42387.750000,1.267971e+07
105,walnut_creek,8.123971e+09,7.517505e+07,32806,25176019,9.360572e+09,5.464893e+07,87208.000000,1.416764e+07
106,windsor,2.121892e+09,2.244781e+07,9731,5600563,4.036407e+09,1.933141e+07,8475.000000,4.499609e+06
107,woodside,1.447092e+09,6.119433e+06,2166,987934,2.110878e+09,1.386529e+07,1944.000000,0.000000e+00


,jurisdiction,improvement_value_10,building_sqft_10,res_units_10,non_res_sqft_10,improvement_value_18,building_sqft_18,res_units_18,non_res_sqft_18,improvement_value_diff,improvement_value_diff_pct,building_sqft_diff,building_sqft_diff_pct,res_units_diff,res_units_diff_pct,non_res_sqft_diff,non_res_sqft_diff_pct
0,alameda,"5,370,707,105","60,694,060","32,056","20,015,176","7,922,533,899","48,206,320","30,080","9,642,204","2,551,826,794",0.475138,"-12,487,740",-0.205749,"-1,976",-0.061642,"-10,372,972",-0.518255
1,albany,"1,053,810,393","10,010,827","7,779","2,539,429","1,604,529,135","9,004,613","5,665","2,264,891","550,718,742",0.522598,"-1,006,214",-0.100513,"-2,114",-0.271757,"-274,538",-0.108110
2,american_canyon,"304,025,012","19,337,562","5,773","7,620,221","2,047,427,721","16,837,687",434,"6,004,531","1,743,402,709",5.734406,"-2,499,875",-0.129276,"-5,339",-0.924822,"-1,615,690",-0.212027
3,antioch,"5,899,923,469","76,405,424","34,821","17,003,933","8,380,102,414","69,531,547","141,212","10,208,477","2,480,178,945",0.420375,"-6,873,877",-0.089966,"106,391",3.055378,"-6,795,455",-0.399640
4,atherton,"2,533,024,320","8,785,455","2,523","1,239,883","4,298,967,433","23,493,587","2,473",0,"1,765,943,113",0.697168,"14,708,131",1.674146,-50,-0.019818,"-1,239,883",-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,vallejo,"6,281,621,834","87,456,818","44,357","17,703,716","8,560,593,683","115,046,921","42,387","12,679,714","2,278,971,849",0.362800,"27,590,103",0.315471,"-1,969",-0.044395,"-5,024,001",-0.283782
105,walnut_creek,"8,123,970,879","75,175,046","32,806","25,176,019","9,360,571,705","54,648,931","87,208","14,167,637","1,236,600,826",0.152216,"-20,526,114",-0.273044,"54,402",1.658294,"-11,008,381",-0.437257
106,windsor,"2,121,891,594","22,447,814","9,731","5,600,563","4,036,407,172","19,331,413","8,475","4,499,609","1,914,515,578",0.902268,"-3,116,401",-0.138829,"-1,256",-0.129072,"-1,100,954",-0.196579
107,woodside,"1,447,091,700","6,119,432","2,166","987,934","2,110,877,932","13,865,289","1,944",0,"663,786,232",0.458704,"7,745,856",1.265780,-222,-0.102493,"-987,934",-1.000000


In [275]:
writer = pd.ExcelWriter('results/BASIS_b18_20200401.xlsx', engine='xlsxwriter')

ls = []
name = []
dfs = {'county': b_cty, 'jurisdiction': b_jur}
atts = ['improvement_value','building_sqft','res_units','non_res_sqft']
for i in list(dfs.keys()):
    for j in atts:
        result = summaryTable(dfs[i],i,j)
        ls.append(result[1])
        name.append(result[0])

for i in name:
    ls[name.index(i)].to_excel(writer, sheet_name = i)

writer.save()

In [276]:
b18.to_csv('results/b18.csv', index = False)
b10.to_csv('results/b10.csv', index = False)